### Use Tensorflow to create a Convolutional Neural Network

In [0]:
from __future__ import print_function
import os
import numpy as np
import pickle
import tensorflow as tf

from six.moves import range

### Connect with Google Drive to mount the data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Load the data

In [41]:
# First reload the data we generated in Assignment 1
#pickle_file = '/content/gdrive/My Drive/test/notMNIST.pickle'
pickle_file = 'notMNISTtiny.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100, 28, 28) (100,)
Validation set (20, 28, 28) (20,)
Test set (40, 28, 28) (40,)


### Reformat the data for training

In [42]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100, 28, 28, 1) (100, 10)
Validation set (20, 28, 28, 1) (20, 10)
Test set (40, 28, 28, 1) (40, 10)


### Define some functions to calculate accuracy, create a directory to reuse for all the tests below

In [0]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [0]:
def safe_mkdir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass

### Create a basic Convolutional Network
Two convolutional layers with Relu and one Fully Connected layer

In [0]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

### Create a Convolutional Network with Max Pooling

Two convolutional layers with Relu and one Fully Connected layer

In [0]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool1 = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], 
                            strides=[1, 2, 2, 1], padding='VALID')
    conv = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool2 = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], 
                            strides=[1, 2, 2, 1], padding='VALID')
    shape = pool2.get_shape().as_list()
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

### Run the model to train it - reused for the models above

In [0]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.791311
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.467512
Minibatch accuracy: 84.4%
Validation accuracy: 84.5%
Minibatch loss at step 1000: 0.506753
Minibatch accuracy: 84.4%
Validation accuracy: 86.7%
Minibatch loss at step 1500: 0.495117
Minibatch accuracy: 82.8%
Validation accuracy: 87.6%
Minibatch loss at step 2000: 0.493518
Minibatch accuracy: 83.6%
Validation accuracy: 88.3%
Minibatch loss at step 2500: 0.355758
Minibatch accuracy: 88.3%
Validation accuracy: 88.7%
Minibatch loss at step 3000: 0.200726
Minibatch accuracy: 95.3%
Validation accuracy: 89.0%
Test accuracy: 94.3%


### OBSOLETE

In [0]:
'''
Create a convolution + relu layer on the data
'''
def _conv_relu(data, filter_num, filter_size, stride, padding, scope_name):
      
      in_depth = data.shape[-1] # Last dimension of the data is the depth
      out_depth = filter_num

      with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        weight = tf.get_variable('weight', 
                     [filter_size, filter_size, in_depth, out_depth],
                     initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias = tf.get_variable('bias', initializer=tf.zeros(out_depth))
        conv = tf.nn.conv2d(data, weight, strides=[1, stride, stride, 1], padding=padding)
      
      return tf.nn.relu(conv + bias, name=scope.name)        
    
'''
Create a maxpool layer
'''
def _maxpool(data, filter_size, stride, padding='VALID', scope_name='pool'):
      with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        pool = tf.nn.max_pool(data, 
                            ksize=[1, filter_size, filter_size, 1], 
                            strides=[1, stride, stride, 1],
                            padding=padding)
      return pool

'''
Create a fully connected layer
'''
def _fully_connected(inputs, out_dim, scope_name='fc'):
      with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        in_dim = inputs.shape[-1]
        w = tf.get_variable('w', [in_dim, out_dim],
                            initializer=tf.truncated_normal_initializer(stddev=0.1))
        b = tf.get_variable('b', [out_dim],
                            initializer=tf.constant_initializer(0.0))
        out = tf.matmul(inputs, w) + b
        
      return out

'''
Define an accuracy measure
'''
def _accuracy(predictions, labels):
      correct_preds = tf.equal(tf.argmax(predictions, 1), tf.argmax(labels, 1))
      return 100 * tf.reduce_mean(tf.cast(correct_preds, tf.float32))
    
class ConvNet(object):
    def __init__(self, filter_size, filter_num, num_hidden, image_size, num_labels, num_channels):
      self.filter_size = filter_size
      self.filter_num = filter_num
      self.num_hidden = num_hidden
      self.image_size = image_size
      self.num_labels = num_labels
      self.num_channels = num_channels
      self.gstep = tf.Variable(0, dtype=tf.int32, 
                                trainable=False, name='global_step')

    '''
    Prepare the input data
    '''
    def _get_data(self):
      # Input data.
      with tf.name_scope('data'):
        self.X = tf.placeholder(tf.float32, shape=(None, self.image_size, 
                                         self.image_size, self.num_channels))
        self.y = tf.placeholder(tf.float32, shape=(None, self.num_labels))
    
    '''
    Create all the layers of the network
    '''
    def _inference(self):
      conv1 = _conv_relu(self.X, self.filter_num, self.filter_size, 1, 'SAME', 'conv1')
      pool1 = _maxpool(conv1, 2, 2, 'VALID', 'pool1')
      
      conv2 = _conv_relu(pool1, self.filter_num, self.filter_size, 1, 'SAME', 'conv2')
      pool2 = _maxpool(conv2, 2, 2, 'VALID', 'pool2')
      
      shape = pool2.get_shape().as_list()
      reshape = tf.reshape(pool2, [-1, shape[1] * shape[2] * shape[3]])
      
      fc1 = _fully_connected(reshape, self.num_hidden, 'fc1')
      self.logits = _fully_connected(fc1, self.num_labels, 'logits')

    '''
    Define loss function - use softmax cross entropy with logits
    '''
    def _loss(self):
      with tf.name_scope('loss'):
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y, logits=self.logits), name='loss')
      
    '''
    Define Optimiser - Use Gradient Descent to minimize cost
    '''
    def _optimize(self):
      self.optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(self.loss)
      
    '''
    Create summaries to write on TensorBoard
    '''      
    def _summary(self):
      with tf.name_scope('summaries'):
            tf.summary.scalar('loss', self.loss)
            tf.summary.scalar('accuracy', self.accuracy)
            tf.summary.histogram('histogram_loss', self.loss)
            self.summary_op = tf.summary.merge_all()
    
    '''
    Count the number of right predictions in a batch
    '''
    def _eval(self):
      with tf.name_scope('predict'):
        prediction = tf.nn.softmax(self.logits)
        self.accuracy = _accuracy (prediction, self.y)

    def _train_one_epoch(self, session, saver, init, writer, epoch, gstep):
        batch_size = 4
        num_batches = 25
        
        for batch_no in range(num_batches):
          # Pick an offset within the training data, which has been randomized.
          # Note: we could use better randomization across epochs.
          offset = (batch_no * batch_size) % (train_labels.shape[0] - batch_size)

          # Generate a minibatch
          batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
          batch_labels = train_labels[offset:(offset + batch_size), :]

          # Run the optimiser and predictions on the training data
          # Prepare a dictionary telling the session where to feed the minibatch.
          # The key of the dictionary is the placeholder node of the graph to be fed,
          # and the value is the numpy array to feed to it.
          feed_dict = {self.X : batch_data, self.y : batch_labels}
          _, l, train_accuracy, summaries = session.run(
            [self.optimizer, self.loss, self.accuracy, self.summary_op], feed_dict=feed_dict)

          if (batch_no % 5 == 0):
            writer.add_summary(summaries, global_step=gstep)
            
            print('Minibatch loss at batch no %d: %f' % (batch_no, l))
            print('Minibatch accuracy: %.1f%%' % train_accuracy)

            valid_feed_dict = {self.X : valid_dataset, self.y : valid_labels}
            valid_accuracy = session.run(self.accuracy, feed_dict=valid_feed_dict)
            print('Validation accuracy: %.1f%%' % valid_accuracy)
        
          gstep += 1
        
        # Save model state as of this global step number
        saver.save(session, 'checkpoints/convnet_mnist', gstep)
        
        return gstep


    def _eval_once(self, session, init, writer, epoch, gstep):
        test_feed_dict = {self.X : test_dataset, self.y : test_labels}
        test_accuracy, test_summaries = session.run([self.accuracy, self.summary_op], feed_dict=test_feed_dict)
        writer.add_summary(test_summaries, global_step=gstep)
        print('Test accuracy: %.1f%%' % test_accuracy)

    
    def build(self):
        self._get_data()
        self._inference()
        self._loss()
        self._optimize()
        self._eval()
        self._summary()

    def train(self, n_epochs):
      
      # Directory to store checkpoints
      CKPT_DIR = 'checkpoints'
      safe_mkdir(CKPT_DIR)
        
      # Save summary data for Tensorboard visualisation
      LOG_DIR = 'graphs'
      writer = tf.summary.FileWriter(LOG_DIR, graph)
      
      with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print('Initialized')

        # Restore model state from prior checkpoint if it exists
        saver = tf.train.Saver()
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
        if ckpt and ckpt.model_checkpoint_path:
           print('Found checkpoint, Restoring ', ckpt.model_checkpoint_path)
           saver.restore(session, ckpt.model_checkpoint_path)
        
        # Global counter of number of training loops done so far
        gstep = self.gstep.eval()
        
        init = ''
        for epoch in range(n_epochs):
            gstep = self._train_one_epoch(session, saver, init, writer, epoch, gstep)
            self._eval_once(session, init, writer, epoch, gstep)
        
      writer.close()

      

In [0]:
FILTER_SIZE = 5
FILTER_NUM = 16
NUM_HIDDEN = 64
IMAGE_SIZE = 28
NUM_LABELS = 10
NUM_CHANNELS = 1 # grayscale

graph = tf.Graph()

with graph.as_default():
   model = ConvNet(FILTER_SIZE, FILTER_NUM, NUM_HIDDEN, IMAGE_SIZE, NUM_LABELS, NUM_CHANNELS)
   model.build()

model.train(n_epochs=3)

In [0]:
!ls graphs

events.out.tfevents.1545307425.eea806322868
events.out.tfevents.1545307958.eea806322868
events.out.tfevents.1545308395.eea806322868
events.out.tfevents.1545308445.eea806322868


### Create an Improved Convolutional Layer with Max Pool - V1

*   Re-use the complete graph for training, testing and validation data
*   Define a class for the Model
*   Define reusable methods to create each layer of the network
*   Variable Scoping
*   Use epochs during training
*   Save and restore intermediate state
*   Visualisation
*   Use tf.datasets
*   L2 Regularisation and Dropout
*   Add Learning Decay?

In [0]:
'''
Create a convolution + relu layer on the data
'''
def _conv_relu(data, filter_num, filter_size, stride, padding, scope_name):
      
      in_depth = data.shape[-1] # Last dimension of the data is the depth
      out_depth = filter_num

      with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        weight = tf.get_variable('weight', 
                     [filter_size, filter_size, in_depth, out_depth],
                     initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias = tf.get_variable('bias', initializer=tf.zeros(out_depth))
        conv = tf.nn.conv2d(data, weight, strides=[1, stride, stride, 1], padding=padding)
      
      return tf.nn.relu(conv + bias, name=scope.name)        
    
'''
Create a maxpool layer
'''
def _maxpool(data, filter_size, stride, padding='VALID', scope_name='pool'):
      with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        pool = tf.nn.max_pool(data, 
                            ksize=[1, filter_size, filter_size, 1], 
                            strides=[1, stride, stride, 1],
                            padding=padding)
      return pool

'''
Create a fully connected layer
'''
def _fully_connected(inputs, out_dim, activation, scope_name='fc'):
      with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
        in_dim = inputs.shape[-1] # shape of the last dimension
        w = tf.get_variable('w', [in_dim, out_dim],
                            initializer=tf.truncated_normal_initializer(stddev=0.1))
        b = tf.get_variable('b', [out_dim],
                            initializer=tf.constant_initializer(0.0))
        out = tf.matmul(inputs, w) + b
        
        # Add the weight to a collection so regularisation loss can be applied
        # later
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, w)
        
        # Call the activation function if passed in
        if activation:
           out = activation (out)
      
      return out

'''
Define an accuracy measure
'''
def _accuracy(predictions, labels):
      # Predictions is a softmax with one column for the probabilities of each 
      # label. Doing an argmax along axis=1 (ie. column) gives the index of the
      # column with the highest predicted probability. The rows are the samples.
      #
      # Similarly, labels is a one-hot encoding with one column for each label.
      # Doing an argmax along axis=1 (ie. column) gives the index of the column
      # with the correct expected value ie. the column with the non-zero value. The
      # rows are the samples.
      #
      # The tf.equal then compares the column index of the predicted probability 
      # with the column index of the expected value for all samples in the batch, 
      # resulting in a vector where all the correctly predicted samples are 
      # True (or 1) and the incorrect samples are False (or 0)
      #
      # Then, doing a reduce_mean of that vector basically sums up all the 
      # samples with a True (or 1) and divides by the total number of samples,
      # giving the % of correctly predicted samples, which is the accuracy.
      correct_preds = tf.equal(tf.argmax(predictions, 1), tf.argmax(labels, 1))
      return 100 * tf.reduce_mean(tf.cast(correct_preds, tf.float32))
    
class ConvNet(object):
    def __init__(self, filter_size, filter_num, num_hidden, image_size, num_labels, num_channels, batch_size, drop_rate, regul_rate):
      self.filter_size = filter_size
      self.filter_num = filter_num
      self.num_hidden = num_hidden
      self.image_size = image_size
      self.num_labels = num_labels
      self.num_channels = num_channels
      self.batch_size = batch_size

      # Set the default value to 1.0. We use this value for all evaluation with
      # the test and validation data, so it doesn't need to passed in via feed_dict
      # Only during training we pass in a different DROP_RATE value via feed_dict
      self.keep_prob = tf.placeholder_with_default(1.0, shape=(), name='keep_prob')
      self.drop_rate = drop_rate
      
      # L2 regularisation rate
      self.beta_regul = tf.placeholder_with_default(regul_rate, shape=(), name='beta_regul')
      
      self.gstep = tf.Variable(1, dtype=tf.int32, 
                                trainable=False, name='global_step')

    '''
    Prepare the input data
    '''
    def _get_data(self):
      # Input data.
      with tf.name_scope('data'):
        # Set up the training dataset
        train_ds = tf.data.Dataset.from_tensor_slices((train_dataset, train_labels))
        #train_ds = train_ds.shuffle(10000) # if you want to shuffle your data
        train_ds = train_ds.batch(self.batch_size)

        # Set up the validation and test datasets
        valid_ds = tf.data.Dataset.from_tensor_slices((valid_dataset, valid_labels))
        valid_ds = valid_ds.batch(valid_dataset.shape[0])   # Get all the validation data in a single batch
        test_ds = tf.data.Dataset.from_tensor_slices((test_dataset, test_labels))
        test_ds = test_ds.batch(test_dataset.shape[0])   # Get all the test data in a single batch
        
        # Feedable - Set up a single generic iterator, not tied to any dataset
        self.handle = tf.placeholder(tf.string, shape=[])
        iterator = tf.data.Iterator.from_string_handle(
            self.handle, train_ds.output_types, train_ds.output_shapes)
        
        # Use the generic iterator to get generic ops for the data and labels, which are still not
        # tied to any dataset. We will build the model using these generic ops
        self.X, self.y = iterator.get_next()
        
        # Now create three separate iterators for each of the three datasets
        self.train_iter = train_ds.make_initializable_iterator()
        self.valid_iter = valid_ds.make_initializable_iterator()
        self.test_iter = test_ds.make_initializable_iterator()
    
    '''
    Create all the layers of the network
    '''
    def _inference(self):
      # First convolution and max pool layer
      conv1 = _conv_relu(self.X, self.filter_num, self.filter_size, 1, 'SAME', 'conv1')
      pool1 = _maxpool(conv1, 2, 2, 'VALID', 'pool1')
      
      # Second convolution and max pool layer
      conv2 = _conv_relu(pool1, self.filter_num, self.filter_size, 1, 'SAME', 'conv2')
      pool2 = _maxpool(conv2, 2, 2, 'VALID', 'pool2')
      
      # Reshape the output of the convolution layers, by flattening it out
      # so that it can be fed to the fully connected layer
      shape = pool2.get_shape().as_list()
      reshape = tf.reshape(pool2, [-1, shape[1] * shape[2] * shape[3]])
      
      # First fully connected layer viz. hidden layer
      fc1 = _fully_connected(reshape, self.num_hidden, tf.nn.relu, 'fc1')
      
      # Add a dropout layer
      dropout = tf.nn.dropout (fc1, self.keep_prob, name='dropout')
      
      # Second fully connected layer viz. output layer
      self.logits = _fully_connected(dropout, self.num_labels, None, 'logits')

    '''
    Define loss function - use softmax cross entropy with logits
    '''
    def _loss(self):
      with tf.name_scope('loss'):
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y, logits=self.logits), name='loss')
        
        # Get the collection of weights marked earlier for regularisation, apply
        # L2 regularisation to them, and add it to the loss 
        w_regul_collection = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        self.loss += self.beta_regul * tf.reduce_sum ([tf.nn.l2_loss(w) for w in w_regul_collection])   
      
    '''
    Define Optimiser - Use Gradient Descent to minimize cost
    '''
    def _optimize(self):
      self.optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(self.loss)
      
    '''
    Create summaries to write on TensorBoard
    '''      
    def _summary(self):
      with tf.name_scope('summaries'):
            tf.summary.scalar('loss', self.loss)
            tf.summary.scalar('accuracy', self.accuracy)
            tf.summary.histogram('histogram_loss', self.loss)
            self.summary_op = tf.summary.merge_all()
    
    '''
    Count the number of right predictions in a batch
    '''
    def _eval(self):
      with tf.name_scope('predict'):
        prediction = tf.nn.softmax(self.logits)
        self.accuracy = _accuracy (prediction, self.y)

    def _train_one_epoch(self, session, saver, train_iter, valid_iter, writer, epoch, gstep):
        # Initialise the training data and get handles for the training and 
        # validation to pass to the Feedable
        session.run(train_iter.initializer)
        train_handle = session.run(train_iter.string_handle())
        valid_handle = session.run(valid_iter.string_handle())
        
        # We don't need to pre-decide the number of mini-batches. We keep going
        # through the dataset, picking up one batch size at a time. When the
        # data is finished, we get an exception which allows us to exit the loop
        while True:
          try:
            # Run the optimiser and predictions on the training data
            # Set the drop rate since we're in the training phase (we will 
            # not do this for validation and testing)
            _, l, train_accuracy, summaries = session.run(
              [self.optimizer, self.loss, self.accuracy, self.summary_op], feed_dict={self.handle: train_handle, self.keep_prob: self.drop_rate})

            if (gstep % 5 == 0):
              writer.add_summary(summaries, global_step=gstep)

              print('Minibatch loss at epoch %d / step %d: %f' % (epoch, gstep, l))
              print('Minibatch accuracy at epoch %d / step %d: %.1f%%' % (epoch, gstep, train_accuracy))

              # We use a Feedable Iterator (instead of a Reinitialisable one) because
              # of the below logic, where we are evaluating the validation data in
              # the middle of an epoch. If we only evaluated it at the end of the
              # epoch (like we do for the test data), a Reinitialisable Iterator
              # would have been fine.

              # Initialise the validation data
              session.run(valid_iter.initializer)
              valid_accuracy = session.run(self.accuracy, feed_dict={self.handle: valid_handle})
              print('Validation accuracy at epoch %d / step %d: %.1f%%' % (epoch, gstep, valid_accuracy))
 
            gstep += 1

          except tf.errors.OutOfRangeError:
            # Data is over, so exit the loop
            break
          
        # Save model state as of this global step number
        saver.save(session, 'checkpoints/convnet_mnist', gstep)
        
        return gstep


    def _eval_once(self, session, test_iter, writer, epoch, gstep):
        # The `Iterator.string_handle()` method returns a tensor that can be evaluated
        # and used to feed the `handle` placeholder.
        test_handle = session.run(test_iter.string_handle())
        
        # Run one pass over the test data
        session.run(test_iter.initializer)
        test_accuracy, test_summaries = session.run([self.accuracy, self.summary_op], feed_dict={self.handle: test_handle})
        writer.add_summary(test_summaries, global_step=gstep)
        print('Test accuracy at epoch %d : %.1f%%' % (epoch, test_accuracy))
    
    def build(self):
        self._get_data()
        self._inference()
        self._loss()
        self._optimize()
        self._eval()
        self._summary()

    def train(self, n_epochs):
      
      # Directory to store checkpoints
      CKPT_DIR = 'checkpoints'
      safe_mkdir(CKPT_DIR)
        
      # Save summary data for Tensorboard visualisation. Create separate graphs
      # for the training and test data
      LOG_DIR = 'graphs'
      train_writer = tf.summary.FileWriter(LOG_DIR + '/train', graph)
      test_writer = tf.summary.FileWriter(LOG_DIR + '/test', graph)
      
      with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print('Initialized')

        # Restore model state from prior checkpoint if it exists
        saver = tf.train.Saver()
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
        if ckpt and ckpt.model_checkpoint_path:
           print('Found checkpoint, Restoring ', ckpt.model_checkpoint_path)
           saver.restore(session, ckpt.model_checkpoint_path)
        
        # Global counter of number of training loops done so far
        gstep = self.gstep.eval()
        
        for epoch in range(n_epochs):
            # Train one epoch with the training and validation data
            gstep = self._train_one_epoch(session, saver, self.train_iter, self.valid_iter, train_writer, epoch, gstep)
            
            # Then evaluate with the test data
            self._eval_once(session, self.test_iter, test_writer, epoch, gstep)
        
      train_writer.close()
      test_writer.close()


In [52]:
FILTER_SIZE = 5
FILTER_NUM = 16
NUM_HIDDEN = 64
IMAGE_SIZE = 28
NUM_LABELS = 10
NUM_CHANNELS = 1 # grayscale
BATCH_SIZE = 4
DROP_RATE = 0.5
REGUL_RATE = 0.001 # Beta rate for regularisation

graph = tf.Graph()

with graph.as_default():
   model = ConvNet(FILTER_SIZE, FILTER_NUM, NUM_HIDDEN, IMAGE_SIZE, NUM_LABELS, NUM_CHANNELS, BATCH_SIZE, DROP_RATE, REGUL_RATE)
   model.build()

model.train(n_epochs=2)

Initialized
Found checkpoint, Restoring  checkpoints/convnet_mnist-51
INFO:tensorflow:Restoring parameters from checkpoints/convnet_mnist-51
Minibatch loss at epoch 0 / step 5: 0.238748
Minibatch accuracy at epoch 0 / step 5: 100.0%
Validation accuracy at epoch 0 / step 5: 75.0%
Minibatch loss at epoch 0 / step 10: 0.299449
Minibatch accuracy at epoch 0 / step 10: 100.0%
Validation accuracy at epoch 0 / step 10: 70.0%
Minibatch loss at epoch 0 / step 15: 0.225857
Minibatch accuracy at epoch 0 / step 15: 100.0%
Validation accuracy at epoch 0 / step 15: 70.0%
Minibatch loss at epoch 0 / step 20: 0.368412
Minibatch accuracy at epoch 0 / step 20: 100.0%
Validation accuracy at epoch 0 / step 20: 65.0%
Minibatch loss at epoch 0 / step 25: 0.238606
Minibatch accuracy at epoch 0 / step 25: 100.0%
Validation accuracy at epoch 0 / step 25: 65.0%
Test accuracy at epoch 0 : 95.0%
Minibatch loss at epoch 1 / step 30: 0.243519
Minibatch accuracy at epoch 1 / step 30: 100.0%
Validation accuracy at ep

In [0]:
!rm -r graphs

### Results so far



Initialized

Test accuracy at epoch 0 : 93.4%

Test accuracy at epoch 1 : 94.2%

Test accuracy at epoch 2 : 94.6%

Test accuracy at epoch 3 : 94.9%

Test accuracy at epoch 4 : 95.1%

Test accuracy at epoch 5 : 95.2%

### Create an Improved Convolutional Layer with Max Pool - V2

*   Use tf.layers

### Optionally set up Tensorboard and grok for Visualisation

In [0]:
# Set the LOGDIR correctly to use Tensorboard
LOG_DIR = 'graphs'

In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

In [0]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [20]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://39a5ed08.ngrok.io
